In [4]:
import praw
import json
import requests
from datetime import datetime

In [5]:
reddit = praw.Reddit(
    client_id ="NKXa252Fg_275FVBx4IvYA",
    client_secret = "Jh5KzHd0mQqNSydJI9ihVlFn_qTrtQ",
    user_agent = "hatespeechAPI",
    username = "flor_he",
    password = "********")

Version 7.6.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


In [6]:
#looking_for = "Antiwork"
#looking_for = "aznidentity"
#looking_for = "AsianMasculinity"
#looking_for = "BlackPeopleTwitter"
#looking_for = "FemaleDatingStrategy"
#looking_for = "GenZedong"
#looking_for = "HermanCainAward"
#looking_for = "KotakuInAction"
#looking_for = "MensRights"
#looking_for = "TheRedPill"
#looking_for = "WhitePeopleTwitter"
looking_for = "Portugueses"

subreddit = reddit.subreddit(looking_for)

In [7]:
new_posts = subreddit.new(limit=1000)

In [8]:
from tqdm import tqdm
import re
import json
from datetime import datetime

def clean_text(text):
    #remove Markdown links, keeping the link text only
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)
    #remove bold formatting
    text = re.sub(r'\*\*([^*]+)\*\*', r'\1', text)
    #remove italics formatting
    text = re.sub(r'\*([^*]+)\*', r'\1', text)
    #remove spoiler tags
    text = re.sub(r'>!\s?([^<]+)\s?!<', r'\1', text)
    #remove hashtags
    text = re.sub(r'#', '', text)
    #remove specific parentheses content
    text = re.sub(r'\(\d+\)', '', text)
    return text

def format_date(utc_timestamp):
    #convert the UTC timestamp to a human-readable date
    return datetime.utcfromtimestamp(utc_timestamp).strftime('%Y-%m-%d %H:%M:%S')

data = []

#wrap `new_posts` with `tqdm` for a progress bar
for post in tqdm(new_posts, desc="Processing posts"):
    post_data = {
        "Title": clean_text(post.title),
        "ID": post.id,
        "URL": post.url,
        "Author": post.author.name if post.author else "Deleted",
        "Score": post.score,
        "Comment count": post.num_comments,
        "Created": format_date(post.created_utc),
        "Comments": []
    }
    post.comments.replace_more(limit=0)
    for comment in post.comments.list():
        comment_data = {
            "Comment ID": comment.id,
            "Author": comment.author.name if comment.author else "Deleted",
            "Comment Body": clean_text(comment.body),
            "Comment Score": comment.score,
            "Created": format_date(comment.created_utc)
        }
        post_data["Comments"].append(comment_data)

    data.append(post_data)

if not data:
    print("No posts were processed. Check if the 'new_posts' is empty or adjust your criteria.")
else:
    filename = f'reddit_posts_and_comments_subreddit_{looking_for}_with_date_and_authors.json'
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Data successfully written to {filename}")


Processing posts: 993it [13:02,  1.27it/s]


Data successfully written to reddit_posts_and_comments_subreddit_Portugueses_with_date_and_authors.json
